In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
%cd /content/gdrive/My Drive/Multiple
!ls

/content/gdrive/My Drive/Multiple
1-conv-128-nodes-0-dense-1585577447-model-0.8418451547622681-accuracy.h5
1-conv-128-nodes-1-dense-1585583570-model-0.8533772826194763-accuracy.h5
1-conv-128-nodes-2-dense-1585591101-model-0.24711696803569794-accuracy.h5
1-conv-32-nodes-0-dense-1585575165-model-0.8583195805549622-accuracy.h5
1-conv-32-nodes-1-dense-1585581033-model-0.24711696803569794-accuracy.h5
1-conv-32-nodes-2-dense-1585587765-model-0.24711696803569794-accuracy.h5
1-conv-64-nodes-0-dense-1585576055-model-0.8550246953964233-accuracy.h5
1-conv-64-nodes-1-dense-1585582167-model-0.265238881111145-accuracy.h5
1-conv-64-nodes-2-dense-1585588770-model-0.24711696803569794-accuracy.h5
2-conv-128-nodes-0-dense-1585577752-model-0.8220757842063904-accuracy.h5
2-conv-128-nodes-1-dense-1585584237-model-0.8550246953964233-accuracy.h5
2-conv-128-nodes-2-dense-1585591608-model-0.8517298102378845-accuracy.h5
2-conv-32-nodes-0-dense-1585575380-model-0.8533772826194763-accuracy.h5
2-conv-32-nodes-1-den

In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import pickle
import time
import numpy as np
import keras.optimizers
from sklearn.metrics import classification_report

Using TensorFlow backend.


In [0]:
import tensorflow
print(tensorflow.__version__)

1.15.2


In [0]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)


In [0]:
import pickle

pickle_in = open("X_test.pickle","rb")
X_test = pickle.load(pickle_in)

pickle_in = open("X_train.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("Y_test.pickle","rb")
Y_test = pickle.load(pickle_in)

pickle_in = open("Y_train.pickle","rb")
Y_train = pickle.load(pickle_in)

In [0]:
print(X_train.shape)
print(X_test.shape)

(1006, 150, 150, 3)
(607, 150, 150, 3)


In [0]:
dense_layers = [0,1,2]
layer_sizes = [32, 64, 128]
conv_layers = [1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

1-conv-32-nodes-0-dense-1585575148
2-conv-32-nodes-0-dense-1585575148
3-conv-32-nodes-0-dense-1585575148
1-conv-64-nodes-0-dense-1585575148
2-conv-64-nodes-0-dense-1585575148
3-conv-64-nodes-0-dense-1585575148
1-conv-128-nodes-0-dense-1585575148
2-conv-128-nodes-0-dense-1585575148
3-conv-128-nodes-0-dense-1585575148
1-conv-32-nodes-1-dense-1585575148
2-conv-32-nodes-1-dense-1585575148
3-conv-32-nodes-1-dense-1585575148
1-conv-64-nodes-1-dense-1585575148
2-conv-64-nodes-1-dense-1585575148
3-conv-64-nodes-1-dense-1585575148
1-conv-128-nodes-1-dense-1585575148
2-conv-128-nodes-1-dense-1585575148
3-conv-128-nodes-1-dense-1585575148
1-conv-32-nodes-2-dense-1585575148
2-conv-32-nodes-2-dense-1585575148
3-conv-32-nodes-2-dense-1585575148
1-conv-64-nodes-2-dense-1585575148
2-conv-64-nodes-2-dense-1585575148
3-conv-64-nodes-2-dense-1585575148
1-conv-128-nodes-2-dense-1585575148
2-conv-128-nodes-2-dense-1585575148
3-conv-128-nodes-2-dense-1585575148


In [0]:
dense_layers = [0,1, 2]
layer_sizes = [32, 64, 128]
conv_layers = [1, 2, 3]


for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)
            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X_train.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1): 
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())
            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))
                model.add(Dropout(0.33))

            model.add(Dense(4))
            model.add(Activation('softmax'))


            model.compile(loss='sparse_categorical_crossentropy',
              optimizer= "adam",
              metrics=['accuracy'],
              )
            
            #Fit the model
            model.fit(X_train, Y_train,
            batch_size=32,
            epochs=20,
            validation_data=(X_test,Y_test),
            callbacks=[tensorboard,es])
                    
            #Score 
            scores = model.evaluate(X_test, Y_test, verbose=1) 
            print('Test loss:', scores[0])
            print('Test accuracy:', scores[1])
            
            #Save model
            model.save("{}-model-{}-accuracy.h5".format(NAME,scores[1]))
            
            #precision    recall  f1-score   support
            y_pred = model.predict(X_test, batch_size=64, verbose=1)
            y_pred_bool = np.argmax(y_pred, axis=1)
            print(classification_report(Y_test, y_pred_bool))
            print()
            print()  

1-conv-32-nodes-0-dense-1585575165
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 1006 samples, validate on 607 samples
Epoch 1/20
1006/1006 [==============================] - 16s 16ms/sample - loss: 184.4736 - acc: 0.4274 - val_loss: 10.4015 - val_acc: 0.6194
Epoch 2/20
1006/1006 [==============================] - 15s 15ms/sample - loss: 3.1891 - acc: 0.7883 - val_loss: 2.4125 - val_acc: 0.8007
Epoch 3/20
1006/1006 [==============================] - 15s 15ms/sample - loss: 0.4053 - acc: 0.9473 - val_loss: 1.9376 - val_acc: 0.8550
Epoch 4/20
1006/1006 [==============================] - 15s 15ms/sample - loss: 0.2105 - acc: 0.9821 - val_loss: 1.8951 - val_acc: 0.8567
Epoch 5/20
1006/1006 [==============================] - 15s 15ms/sample - loss: 0.0358 - acc: 0.9950 - val_loss: 1.7588 - val_acc: 0.8550
Epoch 6/20
1006/1006 [==============================] - 15s 15ms/sample - loss: 0.0119 - acc: 0.9990 - val_loss: 1.7261 - val_acc: 0.8567
Epoch 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train on 1006 samples, validate on 607 samples
Epoch 1/20
1006/1006 [==============================] - 28s 27ms/sample - loss: 20.4203 - acc: 0.3479 - val_loss: 0.8807 - val_acc: 0.6425
Epoch 2/20
1006/1006 [==============================] - 27s 27ms/sample - loss: 0.8609 - acc: 0.6640 - val_loss: 0.5597 - val_acc: 0.8138
Epoch 3/20
1006/1006 [==============================] - 27s 27ms/sample - loss: 0.5055 - acc: 0.8191 - val_loss: 0.4718 - val_acc: 0.8534
Epoch 4/20
1006/1006 [==============================] - 27s 26ms/sample - loss: 0.3542 - acc: 0.8807 - val_loss: 0.3783 - val_acc: 0.8682
Epoch 5/20
1006/1006 [==============================] - 27s 26ms/sample - loss: 0.2610 - acc: 0.9036 - val_loss: 0.4373 - val_acc: 0.8583
Epoch 6/20
1006/1006 [==============================] - 27s 26ms/sample - loss: 0.1955 - acc: 0.9274 - val_loss: 0.3091 - val_acc: 0.8979
Epoch 7/20
1006/1006 [==============================] - 27s 27ms/sample - loss: 0.1452 - acc: 0.9463 - val_loss: 3.1832 - va

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train on 1006 samples, validate on 607 samples
Epoch 1/20
1006/1006 [==============================] - 57s 57ms/sample - loss: 36.2217 - acc: 0.4702 - val_loss: 0.6999 - val_acc: 0.6985
Epoch 2/20
1006/1006 [==============================] - 56s 55ms/sample - loss: 0.5790 - acc: 0.7952 - val_loss: 0.4845 - val_acc: 0.8402
Epoch 3/20
1006/1006 [==============================] - 56s 56ms/sample - loss: 0.2699 - acc: 0.9066 - val_loss: 0.4174 - val_acc: 0.8418
Epoch 4/20
1006/1006 [==============================] - 56s 56ms/sample - loss: 0.1930 - acc: 0.9384 - val_loss: 0.4216 - val_acc: 0.8814
Epoch 5/20
1006/1006 [==============================] - 56s 56ms/sample - loss: 0.0941 - acc: 0.9702 - val_loss: 0.4394 - val_acc: 0.8962
Epoch 6/20
1006/1006 [==============================] - 56s 55ms/sample - loss: 0.0752 - acc: 0.9702 - val_loss: 0.5135 - val_acc: 0.8633
Epoch 7/20
1006/1006 [==============================] - 56s 55ms/sample - loss: 0.0625 - acc: 0.9811 - val_loss: 0.5902 - va

...

Train on 1006 samples, validate on 607 samples
Epoch 1/20
1006/1006 [==============================] - 29s 29ms/sample - loss: 4.7409 - acc: 0.2853 - val_loss: 1.3714 - val_acc: 0.3278
Epoch 2/20
1006/1006 [==============================] - 28s 28ms/sample - loss: 1.3715 - acc: 0.2654 - val_loss: 1.2793 - val_acc: 0.4135
Epoch 3/20
1006/1006 [==============================] - 28s 28ms/sample - loss: 1.3566 - acc: 0.2922 - val_loss: 1.2802 - val_acc: 0.3970
Epoch 4/20
1006/1006 [==============================] - 28s 28ms/sample - loss: 1.3354 - acc: 0.3241 - val_loss: 1.2201 - val_acc: 0.4020
Epoch 5/20
1006/1006 [==============================] - 28s 28ms/sample - loss: 1.3360 - acc: 0.3320 - val_loss: 1.2136 - val_acc: 0.4300
Epoch 6/20
1006/1006 [==============================] - 28s 28ms/sample - loss: 1.3184 - acc: 0.3370 - val_loss: 1.2092 - val_acc: 0.4399
Epoch 7/20
1006/1006 [==============================] - 28s 28ms/sample - loss: 1.3000 - acc: 0.3638 - val_loss: 1.2310 - val

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train on 1006 samples, validate on 607 samples
Epoch 1/20
1006/1006 [==============================] - 58s 58ms/sample - loss: 20.8886 - acc: 0.3459 - val_loss: 1.1244 - val_acc: 0.5750
Epoch 2/20
1006/1006 [==============================] - 57s 56ms/sample - loss: 1.1250 - acc: 0.5408 - val_loss: 0.9695 - val_acc: 0.6458
Epoch 3/20
1006/1006 [==============================] - 57s 56ms/sample - loss: 0.8865 - acc: 0.6640 - val_loss: 0.6106 - val_acc: 0.8138
Epoch 4/20
1006/1006 [==============================] - 56s 56ms/sample - loss: 0.6110 - acc: 0.7763 - val_loss: 0.5723 - val_acc: 0.8171
Epoch 5/20
1006/1006 [==============================] - 56s 56ms/sample - loss: 0.4718 - acc: 0.8300 - val_loss: 0.5131 - val_acc: 0.8517
Epoch 6/20
1006/1006 [==============================] - 56s 56ms/sample - loss: 0.4294 - acc: 0.8489 - val_loss: 0.5635 - val_acc: 0.8435
Epoch 7/20
1006/1006 [==============================] - 56s 56ms/sample - loss: 0.4816 - acc: 0.8509 - val_loss: 0.5349 - va

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train on 1006 samples, validate on 607 samples
Epoch 1/20
1006/1006 [==============================] - 158s 157ms/sample - loss: 15.5621 - acc: 0.4692 - val_loss: 0.8391 - val_acc: 0.6969
Epoch 2/20
1006/1006 [==============================] - 154s 153ms/sample - loss: 0.7403 - acc: 0.7316 - val_loss: 0.7979 - val_acc: 0.7199
Epoch 3/20
1006/1006 [==============================] - 154s 153ms/sample - loss: 0.6260 - acc: 0.8072 - val_loss: 0.5363 - val_acc: 0.8056
Epoch 4/20
1006/1006 [==============================] - 154s 153ms/sample - loss: 0.3784 - acc: 0.8608 - val_loss: 0.4893 - val_acc: 0.8369
Epoch 5/20
1006/1006 [==============================] - 153s 152ms/sample - loss: 0.3314 - acc: 0.8827 - val_loss: 0.5253 - val_acc: 0.8715
Epoch 6/20
1006/1006 [==============================] - 153s 152ms/sample - loss: 0.2256 - acc: 0.9314 - val_loss: 0.6107 - val_acc: 0.8386
Epoch 7/20
1006/1006 [==============================] - 153s 152ms/sample - loss: 0.2513 - acc: 0.9205 - val_los

In [0]:
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 6970), started 0:00:07 ago. (Use '!kill 6970' to kill it.)

<IPython.core.display.Javascript object>

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# Load the TensorBoard notebook extension
%load_ext tensorboard

TensorFlow is already loaded. Please restart the runtime to change versions.
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
%tensorboard --logdir logs

<IPython.core.display.Javascript object>

In [0]:
%reload_ext tensorboard

In [0]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

notebook.display(port=6006, height=1000)

Known TensorBoard instances:
  - port 6006: logdir logs (started 0:05:41 ago; pid 6814)
Selecting TensorBoard with logdir logs (started 0:05:41 ago; port 6006, pid 6814).


In [0]:
!kill 6814

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# Load the TensorBoard notebook extension
%load_ext tensorboard

In [0]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

notebook.display(port=6006, height=1000)

No known TensorBoard instances running.


<IPython.core.display.Javascript object>